In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

### Replace `path` with directory where your `face_images` were downloaded. Available on [Google drive](https://drive.google.com/file/d/1yCYa3IvsvYTkjkl4bvs96pWQfcgOJpxu/view?usp=sharing)

In [ ]:
import os, random
path = '/home/nonsodono/Documents/face_images'
os.chdir(path)  

### Converting Data to Fastai's `DataBlock` objects.

In [ ]:
!pip3 install -Uqq fastbook
import fastbook
from fastbook import *
from fastai.vision.widgets import *

In [ ]:
faces = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.15, seed=42),
    get_y=parent_label
    )

In [ ]:
dls = faces.dataloaders(path)

In [ ]:
dls.train.show_batch(max_n=6, nrows=1)

In [ ]:
dls.valid.show_batch(max_n=6, nrows=1)

### Data Augmentation

In [ ]:
faces = faces.new(item_tfms=Resize(48), batch_tfms=aug_transforms(mult=2))
dls = faces.dataloaders(path)
dls.train.show_batch(max_n=8, nrows=1, unique=True)

### Training the model using Fastai and `vgg19_bn` pretrained model

In [ ]:
model = cnn_learner(dls, vgg19_bn, metrics=accuracy)
model.fine_tune(10)

In [ ]:
interp = ClassificationInterpretation.from_learner(model)
interp.plot_confusion_matrix(figsize=(7,7))

In [ ]:
interp.plot_top_losses(5, nrows=1)

### Cleaning the data based on loss data.

In [ ]:
dls, idxs = DatasetFormatter().from_toplosses(model)
ImageCleaner(dls, idxs, path)

In [ ]:
# cleaner = ImageClassifierCleaner(model)
# cleaner

In [ ]:
# for idx in cleaner.delete(): cleaner.fns[idx].unlink()
# for idx,cat in cleaner.change(): shutil.move(str(cleaner.fns[idx]), str(path/cat).split('.')[0] + "_fixed.jpg")

### Exporting the model 

In [ ]:
model.export()

In [ ]:
path = Path()
path.ls(file_exts='.pkl')

### Make `inferences` from the loaded model

In [ ]:
model_inf = load_learner(path/'export.pkl')

In [ ]:
# Show classes in the model
model_inf.dls.vocab

In [ ]:
model.show_results()

In [ ]:
### Add widgets to upload and classify example images

In [ ]:
btn_upload = widgets.FileUpload()
btn_run = widgets.Button(description='Classify')
out_pl = widgets.Output()
lbl_pred = widgets.Label()

def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = model_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify)

In [ ]:
btn_upload = widgets.FileUpload()
VBox([widgets.Label('Select your face!'), 
      btn_upload, btn_run, out_pl, lbl_pred])